# Eat Safe, Love

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

!mongoimport --type json -duk_food -cestablishments --jsonArray Resources/establishments.json

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
hygiene_20_query ={"scores.Hygiene": 20}
hygiene_20_establishments = establishments.find(hygiene_20_query)

hygiene_20_count = establishments.count_documents(hygiene_20_query)

first_hygiene_20 = hygiene_20_establishments[0]
pprint(first_hygiene_20)


{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677c9dab66c5940f174e99c9'),
 'geocode': {'latitude': '50.769705',
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            '

In [7]:
# Convert the result to a Pandas DataFrame

hygiene_20_df = pd.DataFrame(list(hygiene_20_establishments))

print(hygiene_20_df.head(10))

                        _id   FHRSID  ChangesByServerID  \
0  677c9dab66c5940f174e99c9   110681                  0   
1  677c9dab66c5940f174e9d4e   612039                  0   
2  677c9dab66c5940f174ea055   730933                  0   
3  677c9dab66c5940f174ea243   172735                  0   
4  677c9dab66c5940f174ea254   172953                  0   
5  677c9dab66c5940f174eabf3   512854                  0   
6  677c9dab66c5940f174eae13  1537089                  0   
7  677c9dab66c5940f174ec33e   155648                  0   
8  677c9dab66c5940f174ec779  1012883                  0   
9  677c9dab66c5940f174ecf93   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532              Golden Palace   
5       

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
london_rating_query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "RatingValue": {"$gte": 4}
}
london_rating_establishments = establishments.find(london_rating_query)

london_rating_count = establishments.count_documents(london_rating_query)

try:
    first_london_rating = next(london_rating_establishments)
    pprint(first_london_rating)
except StopIteration:
    print("No establishments found matching the query.")

No establishments found matching the query.


In [9]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(list(london_rating_establishments))

print(f"Number of rows in the DataFrame: {london_rating_df.shape[0]}")

print(london_rating_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

penang_flavours_coords = {
    "latitude": 51.49014200,
    "longitude": 0.08384000
}

degree_search = 0.01
latitude = penang_flavours_coords["latitude"]
longitude = penang_flavours_coords["longitude"]

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("scores.Hygiene", 1)] 
limit = 5

nearby_establishments = establishments.find(query).sort(sort).limit(limit)

for result in nearby_establishments:
    pprint(result)

In [11]:
# Convert result to Pandas DataFrame
results_list = list(nearby_establishments)
df = pd.DataFrame(results_list)

print(df.head())

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    
    {"$sort": {"count": -1}}
]

result = establishments.aggregate(pipeline)
result_list = list(result)

print(f"Number of Local Authorities with hygiene score of 0: {len(result_list)}")

pprint(result_list[:10])

Number of Local Authorities with hygiene score of 0: 56
[{'_id': 'Thanet', 'count': 3390},
 {'_id': 'Greenwich', 'count': 2646},
 {'_id': 'Maidstone', 'count': 2139},
 {'_id': 'Newham', 'count': 2133},
 {'_id': 'Swale', 'count': 2058},
 {'_id': 'Chelmsford', 'count': 2040},
 {'_id': 'Medway', 'count': 2016},
 {'_id': 'Bexley', 'count': 1821},
 {'_id': 'Southend-On-Sea', 'count': 1758},
 {'_id': 'Tendring', 'count': 1626}]


In [13]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result_list)

print(f"Number of rows in the DataFrame: {df.shape[0]}")

print(df.head(10))


Number of rows in the DataFrame: 56
               _id  count
0           Thanet   3390
1        Greenwich   2646
2        Maidstone   2139
3           Newham   2133
4            Swale   2058
5       Chelmsford   2040
6           Medway   2016
7           Bexley   1821
8  Southend-On-Sea   1758
9         Tendring   1626
